# Installing packages

In [ ]:
!pip install --user -q --progress-bar on torch
!pip install --user -q tensorflow
!pip install --user -q sklearn
!pip install -q transformers
!pip install --user -q tqdm
!pip install gdown
!mkdir -p models

***Warning***: Depending on the runtime used, you might have to restart the kernel in order for the new libraries to be located properly.


# Import libraries

In [ ]:
from numpy import dstack
import pickle
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras

from keras.models import load_model
from keras.utils import to_categorical
from keras import Input, regularizers
from keras.layers import Dense, Dropout

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

import gdown

from transformers import TFBertForSequenceClassification, TFDistilBertModel, TFRobertaForSequenceClassification, TFDistilBertForSequenceClassification

# Dataset

Download tokenized tweets from google drive. If the automatic download fails, please copy-paste the link in the browser, download them, and upload them manually.

In [ ]:
files={
    "attention_masks.pkl": "https://drive.google.com/uc?id=1xxv8TWjycS3xRt-9_QrjuGeCsn5lMjge",
    "sentiments_encoded.pkl": "https://drive.google.com/uc?id=1W4bPBy0AO9RKVa1dTiUwp7KIi3LHTIcQ",
    "token_types.pkl": "https://drive.google.com/uc?id=17aTXUlBWWA7eCy2lb7Zwg7APoe0VVqyY",
    "tweets_encoded.pkl": "https://drive.google.com/uc?id=1ZKrDAtIIf5nXeRkAQ9o0wThwwJMPGylX",
}
for fname, url in files.items():
    gdown.download(url, f"{fname}")

## Load the tokenized tweets

In [ ]:
tweets_encoded, attention_masks, token_types, sentiments_encoded = None, None, None, None
with open("tweets_encoded.pkl", "rb") as f:
    tweets_encoded = pickle.load(f)
with open("attention_masks.pkl", "rb") as f:
    attention_masks = pickle.load(f)
with open("token_types.pkl", "rb") as f:
    token_types = pickle.load(f)
with open("sentiments_encoded.pkl", "rb") as f:
    sentiments_encoded = pickle.load(f)

In [ ]:
tweets, sentiments = [tweets_encoded, attention_masks, token_types], sentiments_encoded

## Helper functions 

In [ ]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def dbert_map_example_to_dict(input_ids, attention_masks, label):
  return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
  }, label

## Make train, validation, and test data
Here we have: 90% train (unused as we have done training in prior notebooks, saving the weights), 5% validation, and 5% test.

In [ ]:
def make_train_data(start_tweets, start_sentiments, dbert=False): 
 
      tweets = start_tweets 
      sentiments =  start_sentiments 
 
 
      ratio = 0.9 
         
      print(start_sentiments[:10]) 
      print(start_sentiments[-10:]) 
         
      pos, neg = sentiments[: sentiments.count(1)], sentiments[- sentiments.count(0):] 
         
      np.random.seed(10)   
      msk1 = np.random.rand(len(pos)) < ratio 
      np.random.seed(11)   
      msk2 = np.random.rand(len(neg)) < ratio 
 
      train_tweets = [[], [], []] 
      train_sentiments = [] 
 
      validation_tweets = [[], [], []] 
      validation_sentiments = [] 
 
      test_tweets = [[], [], []] 
      test_sentiments = [] 
 
      rest_tweets = [[], [], []]  
      rest_sentiments = [] 
 
      for i in range(len(pos)): 
        if msk1[i] == True: 
          train_tweets[0].append(tweets[0][i])  
          train_tweets[1].append(tweets[1][i])  
          train_tweets[2].append(tweets[2][i])  
          train_sentiments.append(sentiments[i]) 
        else: 
          rest_tweets[0].append(tweets[0][i])  
          rest_tweets[1].append(tweets[1][i])  
          rest_tweets[2].append(tweets[2][i])  
          rest_sentiments.append(sentiments[i]) 
             
      for i in range(len(neg)): 
        if msk2[i] == True: 
          train_tweets[0].append(tweets[0][len(pos) + i]) 
          train_tweets[1].append(tweets[1][len(pos) +i])  
          train_tweets[2].append(tweets[2][len(pos) +i])   
          train_sentiments.append(sentiments[len(pos) +i]) 
        else: 
          rest_tweets[0].append(tweets[0][len(pos) +i]) 
          rest_tweets[1].append(tweets[1][len(pos) +i]) 
          rest_tweets[2].append(tweets[2][len(pos) +i])  
          rest_sentiments.append(sentiments[len(pos) +i]) 
 
 
      rest_ratio = 0.5 
      print(rest_sentiments[:10]) 
      print(rest_sentiments[-10:]) 
     
      pos, neg = rest_sentiments[: rest_sentiments.count(1)], rest_sentiments[- rest_sentiments.count(0):] 
      np.random.seed(10) 
      msk1 = np.random.rand(len(pos)) < rest_ratio 
      np.random.seed(11)   
      msk2 = np.random.rand(len(neg)) < rest_ratio 
 
 
      for i in range(len(pos)): 
        if msk1[i] == True: 
          validation_tweets[0].append(rest_tweets[0][i]) 
          validation_tweets[1].append(rest_tweets[1][i]) 
          validation_tweets[2].append(rest_tweets[2][i])  
          validation_sentiments.append(rest_sentiments[i]) 
        else: 
          test_tweets[0].append(rest_tweets[0][i])  
          test_tweets[1].append(rest_tweets[1][i])  
          test_tweets[2].append(rest_tweets[2][i])  
          test_sentiments.append(rest_sentiments[i]) 
 
      for i in range(len(neg)): 
        if msk2[i] == True: 
          validation_tweets[0].append(rest_tweets[0][len(pos) + i])  
          validation_tweets[1].append(rest_tweets[1][len(pos) + i])  
          validation_tweets[2].append(rest_tweets[2][len(pos) + i])  
          validation_sentiments.append(rest_sentiments[len(pos) + i]) 
        else: 
          test_tweets[0].append(rest_tweets[0][len(pos) + i])  
          test_tweets[1].append(rest_tweets[1][len(pos) + i])  
          test_tweets[2].append(rest_tweets[2][len(pos) + i])  
          test_sentiments.append(rest_sentiments[len(pos) + i]) 
 
      print("I have: Train:"+str(len(train_sentiments)) + " Validation:" +str(len(validation_sentiments)) + " Test:" +str(len(test_sentiments)))  
     
      if not dbert:
          train_tweets_ds = None
          print("Train loaded") 
          validation_tweets_ds = tf.data.Dataset.from_tensor_slices((validation_tweets[0], validation_tweets[1],validation_tweets[2], validation_sentiments)).map(map_example_to_dict).shuffle(len(validation_sentiments)) 
          print("Validation loaded") 
          test_tweets_ds = tf.data.Dataset.from_tensor_slices((test_tweets[0], test_tweets[1], test_tweets[2],test_sentiments)).map(map_example_to_dict) 
          print("Test loaded") 
          return (train_tweets_ds, train_sentiments), (validation_tweets_ds, validation_sentiments), (test_tweets_ds, test_sentiments) 
      else:
          print("DBERT")
          train_tweets_ds = None
          print("Train loaded")
          validation_tweets_ds = tf.data.Dataset.from_tensor_slices((validation_tweets[0], validation_tweets[1], validation_sentiments)).map(dbert_map_example_to_dict).shuffle(len(validation_sentiments))
          print("Validation loaded")
          test_tweets_ds = tf.data.Dataset.from_tensor_slices((test_tweets[0], test_tweets[1],test_sentiments)).map(dbert_map_example_to_dict)
          print("Test loaded")
          return (train_tweets_ds, train_sentiments), (validation_tweets_ds, validation_sentiments), (test_tweets_ds, test_sentiments) 

In [ ]:
# The datasets for BERT and RoBERTa
(ds_train_encoded_unb,ts), (ds_val_encoded_unb, vs), (ds_test_encoded_unb, tss) = make_train_data(tweets, sentiments)

In [ ]:
# The dataset for DistilBERT - like the one above but wihtout token types
(dbert_ds_train_encoded_unb,ts), (dbert_ds_val_encoded_unb, vs), (dbert_ds_test_encoded_unb, tss) = make_train_data(tweets, sentiments, dbert=True)

## Batch the datasets

In [ ]:
batch_size = 30

ds_test_encoded = ds_test_encoded_unb.batch(batch_size)
ds_val_encoded = ds_val_encoded_unb.batch(batch_size)
dbert_ds_test_encoded = dbert_ds_test_encoded_unb.batch(batch_size)
dbert_ds_val_encoded = dbert_ds_val_encoded_unb.batch(batch_size)

# Functions for loading the base models

In [ ]:
# The DistilBERT model
def get_dbert_model(): 
    num_classes = 2
    max_length=512
    dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased') 
    inps = Input(shape = (max_length,), dtype='int64', name = 'input_ids') 
    masks= Input(shape = (max_length,), dtype='int64', name = 'attention_mask') 
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:] 
    dense = Dense(512,activation='relu',kernel_regularizer=keras.regularizers.l2(0.01))(dbert_layer) 
    dropout= Dropout(0.5)(dense) 
    pred = Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout) 
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    model._name = 'distil_bert'
    return model

In [ ]:
# All models with the weights which resulted after training in the first notebooks loaded
def load_all_models():
    all_models = list()
    
    print("Loading bert-base")
    bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    bert_model.load_weights("models/bert_b30.h5")
    bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    
    print("Loading distilbert-base-uncased")
    dbert_model = get_dbert_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    dbert_model.load_weights("models/dbert_b30.h5")
    dbert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    
    print("Loading roberta")
    roberta_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 2)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    roberta_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    roberta_model.load_weights("models/roberta_b30.h5")
    return [bert_model, dbert_model, roberta_model]

# Ensamble learning

## Helper functions

In [ ]:
def softmax_all(samples):
    softmaxes = list()
    for x in samples:
        e_x = np.exp(x - np.max(x))
        softmaxes.append(e_x / e_x.sum())
    return np.array(softmaxes)

This section is based on this [tutorial](https://machinelearningmastery.com/stacking-ensemble-for-deep-learning-neural-networks/)

In [ ]:
# create stacked dataset
def stacked_dataset(members, inputX, dbertInputX):
    stackX = None
    print("Stacking ensamble dataset:>")
    for model in members:
        # make prediction
        print(model._name)
        if 'distil' in model._name:
            print("Distil")
            yhat = model.predict(dbertInputX, verbose=1)
        else:
            yhat = model.predict(inputX, verbose=1)
            yhat = yhat[0]
            yhat = softmax_all(yhat)
        print(yhat.shape)
        
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, dataset, dbertDataset, ys):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, dataset, dbertDataset)

    # fit standalone model
    model = LogisticRegression(verbose=True)
    model.fit(stackedX, ys)
    return model

# make a prediction with the stacked model
def stacked_prediction(members, model, inputX, dbertInputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX, dbertInputX)

    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

## Loading base models

In [ ]:
members = load_all_models()

## Fitting Ensamble model

In [ ]:
model = fit_stacked_model(members, ds_val_encoded, dbert_ds_val_encoded, vs)

## Evaluating the performance

In [ ]:
yhat = stacked_prediction(members, model, ds_test_encoded, dbert_ds_test_encoded)

acc = accuracy_score(tss, yhat)
print('Stacked accuracy:' + str(acc))